# LAB 03 Assigment

Dominik Breksa (dominikbreksa@student.agh.edu.pl)

### Imports of libraries and packages

In [2]:
import pint
import scipy
import numpy as np
import open_atmos_jupyter_utils
from matplotlib import pyplot

### Setup of libraries and packages

In [3]:
si = pint.UnitRegistry()
si.setup_matplotlib()
si